In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random
import cvxpy as cp

import sys
sys.path.append('..')

## Linear Unconstrained

In [3]:
# Linear env
from env.dynamics import linear
from env.costs import quadratic
from controller.ilqr import ILQRHelper
from env.gymenv import GymEnv

In [4]:
import random
seed = 1
random.seed(seed)
np.random.seed(seed)
_ = torch.manual_seed(seed)

In [15]:
Nx = 4
Nu = 4
T = 20
RQratio = 1e-2
u_max = 2
u_min = 2
rho = 2

# Define Dynamics
dynamics = linear.sample_linear_dynamics(Nx, Nu, A_norm=1.0)

# Define Cost
cost = quadratic.QuadraticCost(
    torch.eye(dynamics.Nx),
    RQratio * torch.eye(dynamics.Nu)
)

# Define how to sample x0
reset_fn = lambda: torch.randn(dynamics.Nx)

u_max = 10
u_min = -10
u_range = u_max - u_min
nominal_env = GymEnv(dynamics, cost, reset_fn, T, ulim=[u_min, u_max])

In [16]:
## Specify the architecture of networks
from agent.policy import LinearActor
from agent.qnetwork import QuadraticQNetwork

actor_creation_fn = lambda obs_dim, act_dim: LinearActor(obs_dim, act_dim)
qnetwork_creation_fn = lambda obs_dim, act_dim: QuadraticQNetwork(obs_dim, act_dim)

def dual_network_creation_fn(nominal_obs_dim, obs_dim):
    dual_network = nn.Linear(
        nominal_obs_dim, obs_dim - nominal_obs_dim,
        bias=False
    )
    dual_network.weight.data.zero_()
    return dual_network

In [17]:
## Specify how to sample new trajectories
def ref_traj_gen_fn(x0s, T, agent, global_step):
    b, Nx = x0s.shape
    assert b == 1, "Can only handle one initial condition at a time"
    x0 = x0s[0]
    Q = agent.nominal_env.cost_fn.Q
    K = agent.actor.gain.weight.data.cpu().numpy()
    P = agent.qf1.get_P().cpu().numpy()
    P = (P + P.T) / 2

    r = cp.Variable(Nx * (T + 1))
    v = agent.dual_network(torch.tensor(x0, device=agent.device)).detach().cpu().numpy()
    if agent.args.no_dual:
        v = np.zeros_like(v)
    rtilde = r + v
    xr = cp.hstack([x0, rtilde])
    u = K @ xr
    xru = cp.hstack([xr, u])
    objective = cp.quad_form(r, np.kron(np.eye(T+1), Q.numpy()))
    objective += cp.quad_form(xru, cp.psd_wrap(P))
    prob = cp.Problem(cp.Minimize(objective), [])
    try:
        prob.solve()
    except Exception as e:
        print(f"Error occured when solving the QP: {e}")
        print(np.linalg.svd(P))
    return torch.tensor(rtilde.value).float().unsqueeze(0)

In [18]:
## How to find optimal cost
A = nominal_env.dynamics.A.numpy()
B = nominal_env.dynamics.B.numpy()
Q = nominal_env.cost_fn.Q.numpy()
R = nominal_env.cost_fn.R.numpy()
nominal_Nx, Nu = B.shape
T = nominal_env.T

def riccati(A, B, Q, R, T):
    P = [None] * (T + 1)
    K = [None] * (T)
    P[T] = Q  # Final cost
    for t in range(T - 1, -1, -1):
        K[t] = -np.linalg.inv(R + B.T @ P[t + 1] @ B) @ B.T @ P[t + 1] @ A
        P[t] = Q + A.T @ P[t + 1] @ A + A.T @ P[t + 1] @ B @ K[t]
    return P, K
    
nominal_P, nominal_K = riccati(A, B, Q, R, T)
opt_cost_fn = lambda x0: x0 @ (nominal_P[0] @ x0)

In [19]:
## Extra validation: optimal map theta
def optimal_dual_map(A, B, Q, R, T, rho):
    # Compute Theta^* of the system (optimal dual map)
    Nx, Nu = B.shape
    E = np.zeros(((T + 1) * Nx, T * Nu))
    for i in range(T+1):
        for j in range(0, i):
            E[i*Nx:(i+1)*Nx, j*Nu:(j+1)*Nu] = np.linalg.matrix_power(A, i-j-1) @ B
    F = np.vstack([np.linalg.matrix_power(A, i) for i in range(T+1)])
    QQ = np.kron(np.eye(T+1), Q)
    RR = np.kron(np.eye(T), R)
    return torch.tensor( -2 / rho * (-QQ @ E @ np.linalg.pinv(E.T @ QQ @ E + RR) @ E.T @ QQ @ F + F))

theta_star = optimal_dual_map(A, B, Q, R, T, rho)

def diff_theta(agent, actual_traj):
    dual_diff_norm = torch.linalg.norm(
        theta_star - agent.dual_network.weight.data.cpu(), 2
    )
    return dual_diff_norm

In [20]:
## Create agent
from agent.layeredargs import LayeredArgs
from agent.layered import LayeredAgent

args = LayeredArgs(
    seed=1,
    torch_deterministic=True,
    exp_name=f"actor_critic",
    total_timesteps=100_000,
    finetune_steps=10_000,
    learning_starts=2_000, # have one batch before 
    buffer_size=10_000,
    gamma=1., # No discount here
    policy_noise= 0.01 / u_range,
    noise_clip= 0.02 / u_range,
    exploration_noise=0.05,
    learning_rate=1e-3,
    dual_optimizer='SGD',
    dual_learning_rate=1e-2,
    dual_batch_size=5,
    cuda=True,
    rho=rho,
    tau=0.005,
)

agent = LayeredAgent(
    nominal_env,
    actor_creation_fn,
    qnetwork_creation_fn,
    dual_network_creation_fn,
    ref_traj_gen_fn,
    opt_cost_fn,
    [('dual_diff', diff_theta)],
    args
)

In [11]:
agent.learn()

## Linear Constrained

In [12]:
# Linear env
from env.dynamics import linear
from env.costs import quadratic
from controller.ilqr import ILQRHelper
from env.gymenv import GymEnv

In [13]:
import random
seed = 3
random.seed(seed)
np.random.seed(seed)
_ = torch.manual_seed(seed)

In [14]:
Nx = 2
Nu = 2
T = 20
RQratio = 1e-2

# Define Dynamics
dynamics = linear.sample_linear_dynamics(Nx, Nu, A_norm=.995)

# Define Cost
cost = quadratic.QuadraticCost(
    torch.eye(dynamics.Nx),
    RQratio * torch.eye(dynamics.Nu)
)

# Define how to sample x0
reset_fn = lambda: torch.randn(dynamics.Nx)

u_max = 10
u_min = -10
u_range = u_max - u_min
nominal_env = GymEnv(dynamics, cost, reset_fn, T, ulim=[u_min, u_max])

In [15]:
## Specify the architecture of networks
from agent.policy import LinearActor
from agent.qnetwork import QuadraticQNetwork

actor_creation_fn = lambda obs_dim, act_dim: LinearActor(obs_dim, act_dim)
qnetwork_creation_fn = lambda obs_dim, act_dim: QuadraticQNetwork(obs_dim, act_dim)

def dual_network_creation_fn(nominal_obs_dim, obs_dim):
    dual_network = nn.Sequential(
        nn.Linear(nominal_obs_dim, 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, obs_dim - nominal_obs_dim),
    )
    dual_network[-1].weight.data.mul_(0.1)
    dual_network[-1].bias.data.mul_(0.01)
    return dual_network

In [16]:
## Specify how to sample new trajectories
def ref_traj_gen_fn(x0s, T, agent, global_step):
    b, Nx = x0s.shape
    assert b == 1, "Can only handle one initial condition at a time"
    x0 = x0s[0]
    Q = agent.nominal_env.cost_fn.Q
    K = agent.actor.gain.weight.data.cpu().numpy()
    P = agent.qf1.get_P().cpu().numpy()
    P = (P + P.T) / 2

    r = cp.Variable(Nx * (T + 1))
    v = agent.dual_network(torch.tensor(x0, device=agent.device)).detach().cpu().numpy()
    if agent.args.no_dual:
        v = np.zeros_like(v)
    rtilde = r + v
    xr = cp.hstack([x0, rtilde])
    u = K @ xr
    xru = cp.hstack([xr, u])
    objective = cp.quad_form(r, np.kron(np.eye(T+1), Q.numpy()))
    objective += cp.quad_form(xru, cp.psd_wrap(P))
    prob = cp.Problem(
        cp.Minimize(objective), [r[2:] >= -0.05]
    )
    try:
        prob.solve()
    except Exception as e:
        print(f"Error occured when solving the QP: {e}")
        print(np.linalg.svd(P))
    return torch.tensor(rtilde.value).float().unsqueeze(0)

In [17]:
A = nominal_env.dynamics.A.numpy()
B = nominal_env.dynamics.B.numpy()
Q = nominal_env.cost_fn.Q.numpy()
R = nominal_env.cost_fn.R.numpy()
nominal_Nx, Nu = B.shape
T = nominal_env.T

def opt_cost_fn(x0):
    x_opt = cp.Variable((T+1, B.shape[0]))
    u_opt = cp.Variable((T, B.shape[1]))
    objective = 0
    constr = [x_opt[0] == x0]
    for t in range(T):
        objective += cp.quad_form(x_opt[t], Q) + cp.quad_form(u_opt[t], R)
        constr += [
            x_opt[t+1] == A @ x_opt[t] + B @ u_opt[t]
        ]
    constr += [x_opt[1:] >= -0.05]
    prob = cp.Problem(cp.Minimize(objective), constr)
    prob.solve()
    return objective.value

In [18]:
## Extra validation: optimal map theta
def constr_violation(agent, actual_traj):
    return (-0.05 - actual_traj[:, 2:]).clip(0, torch.inf).mean().item()

In [19]:
## Create agent
from agent.layeredargs import LayeredArgs
from agent.layered import LayeredAgent

args = LayeredArgs(
    seed=1,
    torch_deterministic=True,
    exp_name=f"actor_critic",
    total_timesteps=150_000,
    finetune_steps=250_000,
    learning_starts=2_000, # have one batch before 
    buffer_size=10_000,
    gamma=1., # No discount here
    policy_noise=0.01 / u_range,
    noise_clip=0.02 / u_range,
    exploration_noise=0.01 / u_range,
    learning_rate=3e-3,
    dual_optimizer='Adam',
    dual_learning_rate=3e-4,
    dual_batch_size=40,
    cuda=True,
    rho=2,
    tau=0.005,
    no_dual=False
)

agent = LayeredAgent(
    nominal_env,
    actor_creation_fn,
    qnetwork_creation_fn,
    dual_network_creation_fn,
    ref_traj_gen_fn,
    opt_cost_fn,
    [('val_constr_vio', constr_violation)],
    args
)

In [ ]:
agent.learn()

## Unicycle

In [21]:
from env.dynamics import unicycle
from env.costs import unicyclecost

In [22]:
u_max = 10
u_min = -10
u_range = u_max - u_min
RQratio = 1e-4
T = 20
dt = 0.1

# Define Dynamics
dynamics = unicycle.UnicycleDynamics(dt=dt)

# Define Cost
cost = quadratic.QuadraticCost(
    torch.diag(torch.tensor([1.,1.,0.,0.])),
    RQratio * torch.eye(dynamics.Nu)
)
tracking_mask = cost.Q.diag() != 0

# Define how to sample x0
def unicycle_reset_fn(sigma_x=1, sigma_theta=0.2):
    x = torch.randn(4) * sigma_x
    x[:2] = x[:2] / torch.linalg.norm(x[:2]) # Normalize the starting position to the unit circle
    x[2] = 0
    x[3] = torch.atan2(-x[1], -x[0]) + torch.randn(1) * sigma_theta
    return x
    
#reset_fn = lambda: torch.randn(dynamics.Nx) / 5
reset_fn = lambda: unicycle_reset_fn()

nominal_env = GymEnv(dynamics, cost, reset_fn, T, ulim=[u_min, u_max])

In [23]:
## Specify the architecture of networks
from agent.policy import TransformerUnicycleActor
from agent.qnetwork import UnicycleQNetwork

actor_creation_fn = lambda obs_dim, act_dim: TransformerUnicycleActor(None, obs_dim, act_dim, u_max, u_min)
qnetwork_creation_fn = lambda obs_dim, act_dim: UnicycleQNetwork(obs_dim, act_dim)

def dual_network_creation_fn(nominal_obs_dim, obs_dim):
    dual_var_dim = int((obs_dim - nominal_obs_dim))
    dual_network = nn.Sequential(
        nn.Linear(nominal_obs_dim, dual_var_dim),
        nn.ReLU(),
        nn.Linear(dual_var_dim, dual_var_dim),
    )
    dual_network[-1].weight.data.mul_(0.01)
    dual_network[-1].bias.data.mul_(0.01)
    return dual_network

In [24]:
## Specify how to sample new trajectories
def interpolate(x, n, noise_std):
    steps = np.linspace(0, 1, n, endpoint=True).reshape(-1, 1)
    interpolated_vectors = x * (1 - steps)
    interpolated_vectors += np.random.randn(*interpolated_vectors.shape) * noise_std
    return torch.tensor(interpolated_vectors.flatten()).float()

#TODO: implementation below assumes that we only have ONE x0 per batch input
def ref_traj_gen_fn(x0s, T, agent, global_step):
    if global_step < 100_000:
        ref = torch.vstack([interpolate(
            xx[agent.validation_env.tracking_mask], T + 1, global_step / 100_000 * 0.03
        ) for xx in x0s])
        return ref
    else:
        x0 = torch.tensor(x0s[0], device=agent.device)
        delta = torch.nn.Parameter(torch.zeros(2 * (T+1), device=agent.device))
        ref_optimizer = torch.optim.SGD([delta], lr=3e-2)

        with torch.no_grad():
            v = agent.dual_network(x0)
            if agent.args.no_dual:
                v.zero_()
    
        steps = torch.linspace(0, 1, T + 1).reshape(-1, 1).to(agent.device)
        nominal = (x0[:2] * (1 - steps)).flatten()
    
        for it in range(agent.args.num_opt_steps):
            ref_optimizer.zero_grad()
            r = nominal + delta
            rtilde = r + v
            xr_tilde = torch.cat([x0, rtilde]).unsqueeze(0)
            u = agent.actor(xr_tilde)
            #TODO: This following line assumes that the state cost has Q=(1, 1, 0, 0)
            cost = -agent.qf1(xr_tilde, u)[0][0] + (r * r).sum()
            cost.backward()
            ref_optimizer.step()
        return rtilde.data.unsqueeze(0)

In [25]:
# optimal cost function
opt_cost_fn = lambda x0: 1

In [26]:
## Create agent
from agent.layeredargs import LayeredArgs
from agent.layered import LayeredAgent

args = LayeredArgs(
    seed=1,
    torch_deterministic=True,
    exp_name=f"actor_critic",
    total_timesteps=500_000,
    finetune_steps=10_000,
    learning_starts=2_000, # have one batch before 
    buffer_size=60_000,
    gamma=1., # No discount here
    policy_noise=1e-3,
    noise_clip=1e-2,
    exploration_noise=4e-2,
    learning_rate=1e-3,
    dual_learning_rate=5e-3,
    dual_batch_size=60,
    cuda=True,
    rho=5,
    tau=0.005,
    num_opt_steps=25,
    dual_optimizer='SGD',
)

agent = LayeredAgent(
    nominal_env,
    actor_creation_fn,
    qnetwork_creation_fn,
    dual_network_creation_fn,
    ref_traj_gen_fn,
    opt_cost_fn,
    [],
    args,
    tracking_mask,
    warmstart_steps=100_000
)

In [ ]:
agent.learn()

In [ ]:
# Evaluate nominal cost of the learned controller
def sim_forward(td3agent, T):
    x, _ = td3agent.envs.reset()
    tc = 0
    xtraj, utraj = [x], []
    for t in range(T):
        with torch.no_grad():
            u = td3agent.actor(torch.tensor(x)).numpy()
        x_, r, term, _, info = td3agent.envs.step(u)
        x = x_
        tc += r
        xtraj.append(x)
        utraj.append(u)
    xtraj = np.vstack(xtraj)
    utraj = np.vstack(utraj)
    return (
                np.vstack([xtraj[:-1], info['final_observation'][0]]),
                utraj,
                info['final_info'][0]['nominal_return'],
                tc
           )

num_samples = 200
ret = []
agent.to('cpu')
for _ in range(num_samples):
    xtraj, utraj, r, tracking_cost = sim_forward(agent, T)
    ret.append(r)
np.mean(ret)

In [84]:
# Find the ILQR baseline
from controller.ilqr import ILQRHelper
ilqrsolver = ILQRHelper(nominal_env)
ilqrsolver.T = nominal_env.T

num_samples = 50
per_sample_restart = 1
costs = []
for i in range(num_samples):
    print(i, end=',')
    x0, _ = nominal_env.reset()
    c = []
    for _ in range(per_sample_restart):
        xtraj, utraj = ilqrsolver.solve(x0, max_iter=30)
        c.append(
            (xtraj[:-1, :2] * xtraj[:-1, :2]).sum() + (utraj * utraj).sum() * RQratio
        )
    costs.append(np.min(c))
np.mean(costs), np.median(costs)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,

(8.898812, 3.638308)